In [ ]:
!pip install -U -q pydrive google-colab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive

# Authenticate and mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# File path
file_path = '/content/drive/MyDrive/data_set_emotion/Annotated data/normalized_data26.12.xlsx'

# Load the Excel file
annotated_data = pd.read_excel(file_path)

# Display the first few rows of the data
print(annotated_data.head())

   Index               Artist         Song  \
0      0  Thisara Weerasinghe  Ansathu Oba   
1      1  Thisara Weerasinghe  Ansathu Oba   
2      2  Thisara Weerasinghe  Ansathu Oba   
3      3  Thisara Weerasinghe  Ansathu Oba   
4      4  Thisara Weerasinghe  Ansathu Oba   

                                             Comment Judge_1_Emotion  \
0  බොහොම ස්තූතී ආදරණීය ප්‍රතිචාරයන්ට..  Channel එ...       Satisfied   
1  ජීවිතේ මේ මොහොත විතරයි...අපිට අපිවත් හිමි නෑ.....         Content   
2             මම නුඹට අයිතියක් නැතුවම ප්‍රේම කරමි ❤🥺            Glad   
3  මේ සින්දු ලස්සනයි එහෙමයි කියලා තමන්ගේ ගෑණි හරි...          Serene   
4                                 ඇත්තටම ලස්සනයි ඒක🥰           Happy   

  Judge_2_Emotion Judge_3_Emotion  Judge_1_Valence  Judge_1_Arousal  \
0       Satisfied       Satisfied             0.52            -0.25   
1         Pleased           Happy             0.60            -0.30   
2            Glad            Glad             0.60             0.40   
3     

In [ ]:
import re
# Convert the 'Comment' column to string, if not already done
annotated_data['Cleaned_Comment'] = annotated_data['Comment'].astype(str)

# Replace emojis with blanks
def replace_emojis_and_dots(text):
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # Emoticons
        "\U0001F300-\U0001F5FF"  # Miscellaneous Symbols and Pictographs
        "\U0001F680-\U0001F6FF"  # Transport and Map Symbols
        "\U0001F700-\U0001F77F"  # Alchemical Symbols
        "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        "\U0001FA00-\U0001FA6F"  # Chess Symbols
        "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        "\U00002702-\U000027B0"  # Dingbats
        "\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)

    text = text.replace('…', ' ')

    # Replace dots with a blank space
    #text = text.replace('.', ' ')
    text = re.sub(r'\.+', ' ', text)

    return text

# Apply the function to the 'Comment' column
# Apply the function to the 'Comment' column
annotated_data['Cleaned_Comment'] = annotated_data['Cleaned_Comment'].apply(replace_emojis_and_dots)

annotated_data['Cleaned_Comment']


,Cleaned_Comment
0,බොහොම ස්තූතී ආදරණීය ප්‍රතිචාරයන්ට Channel එක...
1,ජීවිතේ මේ මොහොත විතරයි අපිට අපිවත් හිමි නෑ ජීව...
2,මම නුඹට අයිතියක් නැතුවම ප්‍රේම කරමි
3,මේ සින්දු ලස්සනයි එහෙමයි කියලා තමන්ගේ ගෑණි හරි...
4,ඇත්තටම ලස්සනයි ඒක
...,...
63410,@inzamamul ෆෙර්දෞස් &lt;බ්\u200dර&gt;මය් හෙඅර්...
63411,එච්ධ්චර බලන්න පිස්සුද ව්හොට්ටෝ
63412,අපිත
63413,@මදුෂානි වක් මෝඩ යකෙක් උබ


In [ ]:
import os
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
import pickle
import torch

In [ ]:
# Set save path for models and checkpoints
save_path = '/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/YouTube_Comments'
os.makedirs(save_path, exist_ok=True)
progress_path = os.path.join(save_path, "progress_checkpoint.pkl")

In [ ]:
# Load your data
# Ensure 'data' contains 'cleaned_comment' and 'label' columns.
# Replace this part with your actual data-loading code
# Example:
# data = pd.read_csv('/path/to/your/data.csv')

# Check if a progress checkpoint exists
if os.path.exists(progress_path):
    print("Resuming from the last checkpoint...")
    with open(progress_path, 'rb') as f:
        checkpoint = pickle.load(f)
    processed_embeddings = checkpoint['processed_embeddings']
    results = checkpoint['results']
else:
    print("Starting fresh...")
    processed_embeddings = []
    results = []

Starting fresh...


Generate Word2Vec Embeddings

In [ ]:
# Generate Word2Vec embeddings if not already processed
if 'word2vec_vector' not in processed_embeddings:
    print("Training Word2Vec model...")
    tokenized_comments = annotated_data['Cleaned_Comment'].apply(lambda x: x.split())
    word2vec_model = Word2Vec(
        sentences=tokenized_comments,
        vector_size=300,
        window=10,
        min_count=1,
        sg=1,
        workers=4
    )
    annotated_data['word2vec_vector'] = annotated_data['Cleaned_Comment'].apply(
        lambda x: np.mean([word2vec_model.wv[word] for word in x.split() if word in word2vec_model.wv], axis=0)
        if x else np.zeros(300)
    )
    processed_embeddings.append('word2vec_vector')
    with open(progress_path, 'wb') as f:
        pickle.dump({'processed_embeddings': processed_embeddings, 'results': results}, f)

Training Word2Vec model...


Generate fastText Embeddings

In [ ]:
!pip install fasttext



In [ ]:
from huggingface_hub import hf_hub_download
import fasttext

try:
    fasttext_model = model = fasttext.load_model(hf_hub_download("facebook/fasttext-vi-vectors", "model.bin"))
    print("fastText model loaded successfully.")
except Exception as e:
    print(f"Failed to load the fastText model: {e}")
    raise


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.bin:   0%|          | 0.00/7.24G [00:00<?, ?B/s]

fastText model loaded successfully.


In [ ]:
# Generate fastText embeddings if not already processed
if 'fasttext_vector' not in processed_embeddings:
    print("Generating fastText embeddings...")
    def get_fasttext_vector(comment, model):
        words = comment.split()
        vectors = [model.get_word_vector(word) for word in words]
        return np.mean(vectors, axis=0) if vectors else np.zeros(300)
    annotated_data['fasttext_vector'] = annotated_data['Cleaned_Comment'].apply(lambda x: get_fasttext_vector(x, fasttext_model))
    processed_embeddings.append('fasttext_vector')
    with open(progress_path, 'wb') as f:
        pickle.dump({'processed_embeddings': processed_embeddings, 'results': results}, f)

Generating fastText embeddings...


Generate SinBERT Embeddings

In [ ]:
# Generate SinBERT embeddings if not already processed
if 'sinbert_vector' not in processed_embeddings:
    print("Generating SinBERT embeddings...")
    tokenizer = AutoTokenizer.from_pretrained("NLPC-UOM/SinBERT-large")
    sinbert_model = AutoModel.from_pretrained("NLPC-UOM/SinBERT-large")

    def get_sinbert_vector(comment, tokenizer, model):
        inputs = tokenizer(comment, return_tensors="pt", truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        hidden_states = outputs.last_hidden_state
        mask = inputs["attention_mask"].unsqueeze(-1).expand(hidden_states.size())
        sum_hidden_states = torch.sum(hidden_states * mask, dim=1)
        sum_mask = torch.sum(mask, dim=1)
        avg_hidden_states = sum_hidden_states / sum_mask
        return avg_hidden_states.squeeze().numpy()

    annotated_data['sinbert_vector'] = annotated_data['Cleaned_Comment'].apply(lambda x: get_sinbert_vector(x, tokenizer, sinbert_model))
    processed_embeddings.append('sinbert_vector')
    with open(progress_path, 'wb') as f:
        pickle.dump({'processed_embeddings': processed_embeddings, 'results': results}, f)

Generating SinBERT embeddings...


config.json:   0%|          | 0.00/722 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.50M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.15M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/504M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at NLPC-UOM/SinBERT-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/504M [00:00<?, ?B/s]

Generate Sentence Embeddings

In [ ]:
# Generate Sentence Embeddings if not already processed
if 'sentence_embedding' not in processed_embeddings:
    print("Generating Sentence Embeddings...")
    sentence_model = SentenceTransformer('all-MiniLM-L6-v2')
    annotated_data['sentence_embedding'] = annotated_data['Cleaned_Comment'].apply(lambda x: sentence_model.encode(x))
    processed_embeddings.append('sentence_embedding')
    with open(progress_path, 'wb') as f:
        pickle.dump({'processed_embeddings': processed_embeddings, 'results': results}, f)

Generating Sentence Embeddings...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Assuming 'annotated_data' is a pandas DataFrame
# Define the file paths
csv_file_path = '/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/YouTube_Comments/annotated_data.csv'
excel_file_path = '/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/YouTube_Comments/annotated_data.xlsx'

# Save to CSV
annotated_data.to_csv(csv_file_path, index=False)

# Save to Excel
annotated_data.to_excel(excel_file_path, index=False)

print(f"Data saved as CSV at {csv_file_path}")
print(f"Data saved as Excel at {excel_file_path}")

Data saved as CSV at /content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/YouTube_Comments/annotated_data.csv
Data saved as Excel at /content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/YouTube_Comments/annotated_data.xlsx


In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import joblib
import json

# ✅ Paths
model_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/YouTube_Comments/MLP_model_2.keras"
config_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/YouTube_Comments/MLP_model_2.keras/config.json"
metadata_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/YouTube_Comments/MLP_model_2.keras/metadata.json"
label_encoder_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/YouTube_Comments/label_encoder.pkl"
new_data_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/YouTube_Comments/annotated_data.xlsx"
output_predictions_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/YouTube_Comments/predictions.csv"

# ✅ Load Model
model = tf.keras.models.load_model(model_path)
print("✅ Model Loaded Successfully!")

# ✅ Load Label Encoder
label_encoder = joblib.load(label_encoder_path)

# ✅ Load and Preprocess New Data
df_new = pd.read_excel(new_data_path).dropna()

# ✅ Parsing Function
def parse_vector(vector_str, expected_dim=768):
    if isinstance(vector_str, str):
        try:
            vec = np.array([float(i) for i in vector_str.strip("[]").split()], dtype=np.float32)
            return np.pad(vec, (0, max(0, expected_dim - len(vec))))[:expected_dim]
        except ValueError:
            pass
    return np.zeros(expected_dim, dtype=np.float32)

# ✅ Convert embeddings into arrays
embedding_features = ['word2vec_vector', 'fasttext_vector', 'sentence_embedding', 'sinbert_vector']
for feature in embedding_features:
    df_new[feature] = df_new[feature].map(lambda x: parse_vector(x, 768))

# ✅ Stack feature vectors
X_new = np.stack(df_new[embedding_features].apply(lambda row: np.column_stack(row.values), axis=1).values)
X_new_flattened = X_new.reshape(X_new.shape[0], -1)

# ✅ Make Predictions
y_pred_prob = model.predict(X_new_flattened)  # Probability scores
y_pred = (y_pred_prob > 0.5).astype(int)  # Convert to binary labels

# ✅ Decode Predictions
df_new['predicted_label'] = label_encoder.inverse_transform(y_pred.flatten())

# ✅ Save Predictions
df_new.to_csv(output_predictions_path, index=False)
print(f"✅ Predictions saved to {output_predictions_path}")


In [ ]:
!pip install -U tensorflow keras
import tensorflow as tf
print(tf.__version__)  # Check if it's the latest


2.18.0


In [ ]:
import tensorflow as tf

model_keras_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/YouTube_Comments/MLP_model_3.keras"
model_h5_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/YouTube_Comments/MLP_model_3.h5"

try:
    print("🔄 Attempting to load .keras model without compilation...")
    model = tf.keras.models.load_model(model_keras_path, compile=False)

    print("✅ Model loaded successfully!")

    # ✅ Convert to .h5 format
    model.save(model_h5_path, save_format="h5")
    print(f"✅ Model converted and saved as {model_h5_path}")

except Exception as e:
    print("❌ Error loading .keras model:", e)


🔄 Attempting to load .keras model without compilation...
❌ Error loading .keras model: Error when deserializing class 'InputLayer' using config={'batch_shape': [None, 3072], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer_3'}.

Exception encountered: Unrecognized keyword arguments: ['batch_shape']


In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import joblib

# ✅ Paths (Updated for MLP_model_2)
model_h5_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/YouTube_Comments/MLP_model_2.h5"
label_encoder_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/YouTube_Comments/label_encoder.pkl"
new_data_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/YouTube_Comments/annotated_data.xlsx"
output_predictions_path = "/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/YouTube_Comments/predictions.csv"

# ✅ Load Model from `.h5`
model = tf.keras.models.load_model(model_h5_path)
print("✅ Model Loaded Successfully from .h5!")

# ✅ Load Label Encoder
label_encoder = joblib.load(label_encoder_path)

# ✅ Load and Preprocess New Data
df_new = pd.read_excel(new_data_path).dropna()

# ✅ Parsing Function
def parse_vector(vector_str, expected_dim=768):
    if isinstance(vector_str, str):
        try:
            vec = np.array([float(i) for i in vector_str.strip("[]").split()], dtype=np.float32)
            return np.pad(vec, (0, max(0, expected_dim - len(vec))))[:expected_dim]
        except ValueError:
            pass
    return np.zeros(expected_dim, dtype=np.float32)

# ✅ Convert embeddings into arrays
embedding_features = ['word2vec_vector', 'fasttext_vector', 'sentence_embedding', 'sinbert_vector']
for feature in embedding_features:
    df_new[feature] = df_new[feature].map(lambda x: parse_vector(x, 768))

# ✅ Stack feature vectors
X_new = np.stack(df_new[embedding_features].apply(lambda row: np.column_stack(row.values), axis=1).values)
X_new_flattened = X_new.reshape(X_new.shape[0], -1)

# ✅ Make Predictions
y_pred_prob = model.predict(X_new_flattened)  # Probability scores
y_pred = (y_pred_prob > 0.5).astype(int)  # Convert to binary labels

# ✅ Decode Predictions
df_new['predicted_label'] = label_encoder.inverse_transform(y_pred.flatten())

# ✅ Save Predictions
df_new.to_csv(output_predictions_path, index=False)
print(f"✅ Predictions saved to {output_predictions_path}")


✅ Model Loaded Successfully from .h5!
1982/1982 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step
✅ Predictions saved to /content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/YouTube_Comments/predictions.csv


In [ ]:
df_new

,Index,Artist,Song,Comment,Judge_1_Emotion,Judge_2_Emotion,Judge_3_Emotion,Judge_1_Valence,Judge_1_Arousal,Judge_2_Valence,...,Std_Judge_2_Valence,Std_Judge_2_Arousal,Std_Judge_3_Valence,Std_Judge_3_Arousal,Cleaned_Comment,word2vec_vector,fasttext_vector,sinbert_vector,sentence_embedding,predicted_label
0,0,Thisara Weerasinghe,Ansathu Oba,බොහොම ස්තූතී ආදරණීය ප්‍රතිචාරයන්ට.. Channel එ...,Satisfied,Satisfied,Satisfied,0.52,-0.25,0.52,...,0.154063,-0.336703,0.158482,-0.336794,බොහොම ස්තූතී ආදරණීය ප්‍රතිචාරයන්ට Channel එක...,"[-0.08008087, 0.43429968, -0.00680197, -0.2077...","[0.0054411134, -0.0063202325, -0.0063125035, 0...","[1.3714868, -0.96511537, 0.65238357, -0.814162...","[-0.0660681, -0.0644375, 0.03815006, -0.040698...",POSITIVE
1,1,Thisara Weerasinghe,Ansathu Oba,ජීවිතේ මේ මොහොත විතරයි...අපිට අපිවත් හිමි නෑ.....,Content,Pleased,Happy,0.60,-0.30,0.60,...,0.224006,-0.069363,0.140929,-0.069436,ජීවිතේ මේ මොහොත විතරයි අපිට අපිවත් හිමි නෑ ජීව...,"[-0.028722236, 0.18478276, -0.092306636, -0.11...","[-0.00014448473, -0.0038836503, 0.00044836223,...","[0.063293055, -0.19856912, 0.42593417, -0.4145...","[0.0010880625, 0.064509355, -0.012029851, -0.0...",POSITIVE
2,2,Thisara Weerasinghe,Ansathu Oba,මම නුඹට අයිතියක් නැතුවම ප්‍රේම කරමි ❤🥺,Glad,Glad,Glad,0.60,0.40,0.60,...,0.224006,0.358382,0.228693,0.358339,මම නුඹට අයිතියක් නැතුවම ප්‍රේම කරමි,"[0.07884229, 0.33124843, -0.3056709, -0.148186...","[0.0025961306, -0.002101397, 0.0017600156, 0.0...","[-0.03643032, -0.31366462, 0.8220717, -0.12696...","[0.061833415, 0.061789818, 0.038210247, 0.0167...",NEGATIVE
3,3,Thisara Weerasinghe,Ansathu Oba,මේ සින්දු ලස්සනයි එහෙමයි කියලා තමන්ගේ ගෑණි හරි...,Serene,Pleased,Happy,0.50,-0.50,0.60,...,0.224006,-0.069363,0.140929,-0.069436,මේ සින්දු ලස්සනයි එහෙමයි කියලා තමන්ගේ ගෑණි හරි...,"[-0.022698194, 0.3060273, -0.042058684, -0.079...","[0.007476813, -0.0052609784, 0.00094434124, 0....","[-0.017085895, -0.06468413, 0.5207233, -0.4606...","[-0.013232362, 0.06302756, 0.013603149, -0.005...",NEGATIVE
4,4,Thisara Weerasinghe,Ansathu Oba,ඇත්තටම ලස්සනයි ඒක🥰,Happy,Pleased,Happy,0.50,0.00,0.60,...,0.224006,-0.069363,0.140929,-0.069436,ඇත්තටම ලස්සනයි ඒක,"[0.03511822, 0.060104314, -0.15778959, -0.1404...","[0.0069657154, -0.007027189, 0.0029487193, 0.0...","[0.8927472, -0.29116082, 1.0343022, -0.3841098...","[0.03817307, 0.041412994, 0.06867824, 0.008860...",POSITIVE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63410,57132,Thisara Weerasinghe,Ansathu Oba,@inzamamul ෆෙර්දෞස් &lt;බ්\u200dර&gt;මය් හෙඅර්...,Delighted,Delighted,Pleased,0.80,0.70,0.80,...,0.398864,0.679191,0.228693,-0.069436,@inzamamul ෆෙර්දෞස් &lt;බ්\u200dර&gt;මය් හෙඅර්...,"[0.15467896, 0.34845135, -0.16687284, -0.17372...","[-0.0015212194, 0.013078285, -0.10568288, 0.03...","[1.4410663, -1.0579543, 0.5841785, -0.78323054...","[-0.047168363, 0.02270466, 0.08271228, 0.02660...",NEGATIVE
63411,57133,Thisara Weerasinghe,Ansathu Oba,එච්ධ්චර බලන්න පිස්සුද ව්හොට්ටෝ,Glad,Glad,Satisfied,0.60,0.40,0.60,...,0.224006,0.358382,0.158482,-0.336794,එච්ධ්චර බලන්න පිස්සුද ව්හොට්ටෝ,"[0.06886965, 0.25113052, 0.01016114, -0.002451...","[0.0017688494, -0.0073710144, -0.0022179147, 0...","[-0.16912407, 0.09121252, 0.08347208, -0.54082...","[0.051806286, 0.05501935, 0.057819396, 0.01012...",NEGATIVE
63412,57134,Thisara Weerasinghe,Ansathu Oba,අපිත,Relaxed,Relaxed,Pleased,0.50,-0.70,0.50,...,0.136577,-0.817916,0.228693,-0.069436,අපිත,"[0.00062084995, -0.0032916248, -0.0032991155, ...","[-0.0014492655, 0.003440294, 0.0014025584, 0.0...","[0.7240734, 0.10312028, 0.7319375, -0.514177, ...","[-0.04159639, -0.024681792, 0.07789305, -0.006...",NEGATIVE
63413,57135,Thisara Weerasinghe,Ansathu Oba,@මදුෂානි වක් මෝඩ යකෙක් උබ,Pleased,Pleased,Happy,0.60,0.00,0.60,...,0.224006,-0.069363,0.140929,-0.069436,@මදුෂානි වක් මෝඩ යකෙක් උබ,"[0.02649709, 0.30580884, -0.05364694, 0.019703...","[-0.004477465, -0.000243547, -0.033809576, 0.0...","[0.013116153, -0.054661714, 1.0947205, -0.7949...","[-0.029051

In [ ]:
# Grouping and summarizing the data
summary_df = df_new.groupby(["Song", "Judge_1_Emotion", "predicted_label"]).size().reset_index(name="predicted_label count")

# Counting total comments per (Song, Emotion)
comment_counts = df_new.groupby(["Song", "Judge_1_Emotion"]).size().reset_index(name="Comment Count")

# Merging both dataframes to get the desired structure
final_df = summary_df.merge(comment_counts, on=["Song", "Judge_1_Emotion"], how="left")

# Reordering columns for clarity
final_df = final_df[["Song", "Judge_1_Emotion", "Comment Count", "predicted_label", "predicted_label count"]]

# Display the final summary
print(final_df)

# Save the summary to the desired location
final_df.to_excel("/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/YouTube_Comments/summary_output_J1.xlsx", index=False)


                   Song Judge_1_Emotion  Comment Count predicted_label  \
0              Ambaruwo           Angry             17        NEGATIVE   
1              Ambaruwo           Angry             17        POSITIVE   
2              Ambaruwo         Annoyed             16        NEGATIVE   
3              Ambaruwo         Annoyed             16        POSITIVE   
4              Ambaruwo         Aroused              7        NEGATIVE   
..                  ...             ...            ...             ...   
680  Unmada Prema Geeya             Sad              3        NEGATIVE   
681  Unmada Prema Geeya             Sad              3        POSITIVE   
682  Unmada Prema Geeya       Satisfied            352        NEGATIVE   
683  Unmada Prema Geeya       Satisfied            352        POSITIVE   
684  Unmada Prema Geeya           Tense              2        NEGATIVE   

     predicted_label count  
0                        8  
1                        9  
2                       

In [ ]:
# Grouping and summarizing the data
summary_df = df_new.groupby(["Song", "Judge_2_Emotion", "predicted_label"]).size().reset_index(name="predicted_label count")

# Counting total comments per (Song, Emotion)
comment_counts = df_new.groupby(["Song", "Judge_2_Emotion"]).size().reset_index(name="Comment Count")

# Merging both dataframes to get the desired structure
final_df = summary_df.merge(comment_counts, on=["Song", "Judge_2_Emotion"], how="left")

# Reordering columns for clarity
final_df = final_df[["Song", "Judge_2_Emotion", "Comment Count", "predicted_label", "predicted_label count"]]

# Display the final summary
print(final_df)

# Save the summary to the desired location
final_df.to_excel("/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/YouTube_Comments/summary_output_J2.xlsx", index=False)

                   Song Judge_2_Emotion  Comment Count predicted_label  \
0              Ambaruwo           Angry             17        NEGATIVE   
1              Ambaruwo           Angry             17        POSITIVE   
2              Ambaruwo         Annoyed             16        NEGATIVE   
3              Ambaruwo         Annoyed             16        POSITIVE   
4              Ambaruwo         Aroused              7        NEGATIVE   
..                  ...             ...            ...             ...   
693  Unmada Prema Geeya             Sad              3        NEGATIVE   
694  Unmada Prema Geeya             Sad              3        POSITIVE   
695  Unmada Prema Geeya       Satisfied            351        NEGATIVE   
696  Unmada Prema Geeya       Satisfied            351        POSITIVE   
697  Unmada Prema Geeya           Tense              2        NEGATIVE   

     predicted_label count  
0                        8  
1                        9  
2                       

In [ ]:
# Grouping and summarizing the data
summary_df = df_new.groupby(["Song", "Judge_3_Emotion", "predicted_label"]).size().reset_index(name="predicted_label count")

# Counting total comments per (Song, Emotion)
comment_counts = df_new.groupby(["Song", "Judge_3_Emotion"]).size().reset_index(name="Comment Count")

# Merging both dataframes to get the desired structure
final_df = summary_df.merge(comment_counts, on=["Song", "Judge_3_Emotion"], how="left")

# Reordering columns for clarity
final_df = final_df[["Song", "Judge_3_Emotion", "Comment Count", "predicted_label", "predicted_label count"]]

# Display the final summary
print(final_df)

# Save the summary to the desired location
final_df.to_excel("/content/drive/MyDrive/data_set_emotion/Other_Emotion_Models/YouTube_Comments/summary_output_J3.xlsx", index=False)

                   Song Judge_3_Emotion  Comment Count predicted_label  \
0              Ambaruwo           Angry             17        NEGATIVE   
1              Ambaruwo           Angry             17        POSITIVE   
2              Ambaruwo         Annoyed             16        NEGATIVE   
3              Ambaruwo         Annoyed             16        POSITIVE   
4              Ambaruwo         Aroused              7        NEGATIVE   
..                  ...             ...            ...             ...   
696  Unmada Prema Geeya             Sad              4        NEGATIVE   
697  Unmada Prema Geeya             Sad              4        POSITIVE   
698  Unmada Prema Geeya       Satisfied            323        NEGATIVE   
699  Unmada Prema Geeya       Satisfied            323        POSITIVE   
700  Unmada Prema Geeya           Tense              2        NEGATIVE   

     predicted_label count  
0                        8  
1                        9  
2                       